In [ ]:
import os
import json
import fsspec

In [ ]:
with fsspec.open(
    "az://carbonplan-retro/classifications.json",
    account_key=os.environ["BLOB_ACCOUNT_KEY"],
    account_name="carbonplan",
    mode="r",
) as f:
    reclassification_weights = json.load(f)

In [ ]:
reclassification_weights["ACR377"]

In [ ]:
from carbonplan_retro.data import cat

retro_json = cat.retro_db_light_json.read()

In [ ]:
from carbonplan_retro.utils import aa_code_to_ss_code

aa_to_ss_d = aa_code_to_ss_code()

In [ ]:
from collections import defaultdict, Counter

In [ ]:
def get_top_species_by_aa(project):
    spcd_per_ss = defaultdict(Counter)
    aa_to_ss_d = aa_code_to_ss_code()
    for assessment_area in project["assessment_areas"]:
        if assessment_area["code"] != 999:
            ss = aa_to_ss_d.get(assessment_area["code"])
            for species in assessment_area["species"]:
                spcd_per_ss[(ss, assessment_area["code"])][species["code"]] += species["fraction"]
        else:
            all_species = {
                species["code"]: species["fraction"] for species in assessment_area["species"]
            }
            return {(ss, 999): all_species for ss in project["supersection_ids"]}

    return spcd_per_ss

In [ ]:
top_species = get_top_species_by_aa(project)

In [ ]:
spcd_to_name = json.load(open("fia_spcd_to_name.json"))

In [ ]:
species_fraction

In [ ]:
records = []
for project in retro_json[:]:
    try:
        top_species = get_top_species_by_aa(project)
        for k, species_fraction in top_species.items():
            ss_id, aa_id = k
            proj_class = reclassification_weights[project["opr_id"]]
            classifications_arr = [
                k for k, v in proj_class[f"({ss_id}, {aa_id})"].items() if v > 0.1
            ]
            species_arr = [
                {"name": spcd_to_name.get(str(k)), "fraction": v}
                for k, v in species_fraction.items()
                if v > 0.1
            ]
            record = {
                "id": project["opr_id"],
                "aa_id": aa_id,
                "ss_id": ss_id,
                "classification": classifications,
                "species": species_arr,
            }
            records.append(record)
    except:
        print(project["opr_id"])

In [ ]:
with open("/home/jovyan/lost+found/reclass_table.json", "w") as f:
    json.dump(records, f)

In [ ]:
records

In [ ]:
%debug

In [ ]:
specspcd_to_name

In [ ]:
for aa_id, species_comp in d.items():
    store = {}
    for species, fraction in species_comp.items():
        if fraction > 0.1:
            store[]